In [ ]:
import sys 
sys.path.append('../../../')
sys.path.append('../../')
sys.path.append('../')

from omegaconf import OmegaConf
from src.core.passive_learning import *
from src.core.auto_labeling import *
from src.utils.logging_utils import * 
from src.data_layer.datasets.dataset_factory import * 
from src.data_layer.dataset_utils import * 
from src.utils.counting_utils import *  #
from src.utils.common_utils import * 
from src.utils.vis_utils import *
#from src.core.run_lib import * 
from src.core.self_training import * 
import copy 
import random 

config_file = '../../configs/self-training/mnist_lenet_base_conf.yaml'
conf = OmegaConf.load(config_file)

logger = get_logger('../../temp/logs/self-training-mnist.log','PL',level=logging.DEBUG)
#logger = get_logger('../../temp/logs/pl.log','PL')

conf['eval'] = 'full'
conf['calib_conf'] = None 

In [ ]:
q_conf = conf["train_pts_query_conf"] 
q_conf['seed_train_size'] = 100
q_conf['max_num_train_pts'] = 100

q_conf['include_pseudo_labeled'] = True 

q_conf['enable_active_querying'] = False 

conf['stopping_criterion'] = "max_epochs"

In [ ]:
auto_lbl_conf = conf["auto_lbl_conf"] 

auto_lbl_conf['threshold_estimation'] = 'val_estimate'
#auto_lbl_conf['threshold_estimation'] = 'fixed'
#auto_lbl_conf['fixed_threshold'] = 0.6


pseudo_lbl_conf = conf["pseudo_lbl_conf"] 

#pseudo_lbl_conf['threshold_estimation'] = 'val_estimate'
pseudo_lbl_conf['threshold_estimation'] = 'fixed'
pseudo_lbl_conf['fixed_threshold'] = 0.95

In [ ]:
set_seed(conf['random_seed'])

dm = DataManager(conf,logger,lib=conf['model_conf']['lib'])

logger.info('Loaded dataset {}'.format(conf['data_conf']['name']))
logger.info(f' std_train_size : {len(dm.ds_std_train)} and  std_val_size: {len(dm.ds_std_val)}')

st = SelfTraining(conf,dm,logger)

st.init()

lst_epoch_out = st.run_al_loop()

logger.info('Self Training Loop Done')